# Evaluer bøker

Gitt en gruppe ord og grupper av dem igjen, hvordan kan vi finne bøker som matcher? Google har sine algoritmer. I dhlab er det en funksjon `nb.wordbag_eval` som henter ut grunndata for å gjøre en evaluering. Sammen med de nye fulltekstindeksene skulle det her gi noen muligheter til å bygge og analyser korpus basert på innholdsord.

Med `wordbag_eval` gis en gruppert liste med ord sammen med et sett URNer. I eksemplet under henter vi ut ord fra galaksene.

In [2]:
import dhlab as dh
import dhlab.api.dhlab_api as api
import dhlab.graph_networkx_louvain as gnl

In [3]:
import words_and_graphs as wg
import numpy as np

## Den her koden vil ikke virke uten en ekstra fil

In [16]:
wg.relaterte_ord('vitenskap', number=4)

,word,score
0,vitenskap,0.376223
1,religion,0.315158
2,filosofi,0.295281
3,kultur,0.294280


## Lag en liste med relateterte ord - temaord

For evaluering, det å finne temaene er det viktig at de er organisert som en python dictionary. På formen 

```{tema_1: liste_av_ord_1, tema_2: liste_2, ...., tema_N: liste_N}```

In [5]:
war = {x:list(wg.relaterte_ord(x).word.values) for x in "våpen kjøretøy båter fly bomber soldater".split()}

In [6]:
print('\n\n'.join([x + ': ' + ', '.join(war[x]) for x in war]))

våpen: våpen, ammunisjon, forsyninger, materiell, krigsutstyr, utstyr, krigsmateriell, sprengstoff, eksplosiver, militærutstyr, personell, våpensystemer, sabotasjemateriell, proviant, våpendeler, våben, militært, rustning, rustninger, forsvarsmateriell

kjøretøy: trafikanter, kjøretøy, maskiner, bygningstekniske, forvalte, føreropplæring, et, anleggsmaskiner, kjoretoy, førere, motorvogn, kjøretoy, kjoretøy, angsten, kollektivtrafikk, hengere, trafikantar, utstyr, fotgjengere, kjoretey

båter: båter, flåter, kanoer, fly, campingvogner, båtutstyr, biler, yachter, motorer, motorkjøretøyer, påhengsmotorer, båtmotorer, redskaper, vannscootere, fiskeredskaper, skip, sykler, småbåter, redningsvester, motorbåter

fly: fly, stridsvogner, krigsskip, marinefartøyer, übåter, ubåter, helikoptre, artilleri, helikoptere, tanks, krigsskib, fartøyer, raketter, flyvere, flygere, bombefly, flydeler, skip, kampfly, pansrede

bomber: bomber, maskingevær, mitraljøser, granater, maskingeværer, giftgass, bomb

### Definer et korpus

Her velger vi et sett med tyve tusen bøker fra 900 serien (historie) for Dewey

In [39]:
corpus = dh.Corpus(doctype="digibok", from_year=1950, to_year = 1990, ddk="9*", lang='nob', limit = 20000)

### Kjør api.evaluate_documents(...)

In [40]:
%%time
docs = api.evaluate_documents(war,urns = list(corpus.frame.urn.values))              

CPU times: user 226 ms, sys: 10.3 ms, total: 236 ms
Wall time: 3.91 s


## Lag en kobling mellom dhlabid og URN

In [41]:
id_dict = corpus.frame[['dhlabid','title']].set_index('dhlabid').astype(str).to_dict()

In [42]:
docs.index = docs.index.astype(int)

In [43]:
#docs['score'] = docs.prod(axis = 1)

## Juster vekter

In [21]:
weights = {x:1 for x in docs.columns}

In [22]:
weights

{'kjøretøy': 1,
 'soldater': 1,
 'båter': 1,
 'fly': 1,
 'våpen': 1,
 'bomber': 1,
 'Weighted Product': 1}

## Juster vekter, eller bare la dem stå som 1

In [33]:
weights = {'kjøretøy': 1,
 'soldater': 1,
 'båter': 1,
 'fly': 1,
 'våpen': 1,
 'bomber': 1}

## Legg til en kolonne med vektede verdier

In [44]:
docs['Weighted Average'] = docs.fillna(0).apply(lambda row: (np.mean([row[col] * weights[col] for col in weights])), axis=1)

In [45]:
(docs
 .fillna(0)
 .rename(index = id_dict['title'])
 .head(20)
)

,kjøretøy,soldater,båter,fly,våpen,bomber,Weighted Average
Gjester ved mitt stambord,648.0,689.0,0.0,0.0,0.0,0.0,222.833333
Etter gull i Alaska,860.0,1688.0,15.0,7.0,14.0,0.0,430.666667
Nordvestpassasjen : beretning om Gjøa-ekspedisjonen 1903-1907,1712.0,2909.0,13.0,15.0,46.0,9.0,784.000000
Jorden rundt på åtte måneder,405.0,1290.0,70.0,56.0,6.0,5.0,305.333333
Hvor demokratisk ? : et essay om det norske politiske system,202.0,572.0,5.0,1.0,0.0,0.0,130.000000
"Sorte , sorgløse Haiti",533.0,1224.0,7.0,5.0,3.0,4.0,296.000000
I kano fra Larvik til Nilens kilder,1021.0,1781.0,16.0,4.0,13.0,3.0,473.000000
Dette er Canada : Krabbe Smith,410.0,1003.0,31.0,10.0,20.0,0.0,245.666667
Veien til deg selv,542.0,1369.0,4.0,3.0,9.0,4.0,321.833333
Tusen og én dag : en bok om India,451.0,868.0,5.0,2.0,4.0,3.0,222.166667


# Juster tallene, relativiser dem

In [46]:
docs_rel = docs*100/docs.sum()

In [47]:
docs_rel['Weighted Average'] = docs_rel.fillna(0).apply(lambda row: (np.mean([row[col] * weights[col] for col in weights])), axis=1)

In [49]:
(docs_rel
 .fillna(0)
 .sort_values(by="Weighted Average", ascending=False)
 .rename(index = id_dict['title'])
 .head(20)
)

,kjøretøy,soldater,båter,fly,våpen,bomber,Weighted Average
Norges sjøkrig 1940-1945 . 3 : Sjøforsvarets kamper og virke på Vestlandet og i Trøndelag i 1940,0.017863,0.020015,0.224598,0.377845,0.177695,0.964453,0.297078
Danmark-Norge mellom stormaktene 1688-1697 : dansk-norsk sjøfart og utenrikspolitikk under den pfalziske arvefølgekrig,0.024169,0.029484,0.704552,0.839656,0.013787,0.035429,0.274513
Århundrets krønike,0.135062,0.141806,0.342990,0.439907,0.199907,0.301146,0.260136
Krigens dagbok : Norge 1940-1945,0.057713,0.062398,0.367365,0.442949,0.211395,0.342479,0.247383
"2. verdenskrig : hvem , hva , hvor",0.033169,0.028449,0.226919,0.703364,0.118718,0.358225,0.244808
Den annen verdenskrig,0.061565,0.062795,0.207187,0.601754,0.232075,0.281463,0.241140
Norges sjøkrig 1940-1945 . 6 : 2 2 : Marinens operasjoner fra baser i Storbritannia m.v. fra januar 1944 til juni 1945,0.021476,0.021872,0.207187,0.292663,0.213693,0.623942,0.230139
Norges sjøkrig 1940-1945 . 2 : Det tyske angrep i Oslofjorden og på Norges sørkyst,0.011624,0.015678,0.163660,0.334646,0.101102,0.618037,0.207458
Operasjonene i Ådalen og i Valdres,0.015272,0.024792,0.179910,0.188010,0.209098,0.594418,0.201917
Norges sjøkrig 1940-1945 . 6 : 1 1 : Marinens operasjoner fra baser i Storbritannia m.v. fra juli 1940 til desember 1943,0.019840,0.021546,0.199062,0.282319,0.071231,0.602291,0.199382
